# Code to create FAT2019 Preprocessed Mel-spectrogram Dataset

In [ ]:
import os
import math
import time
import pickle
import random

import librosa
import numpy as np
import pandas as pd
import PIL
import IPython
import IPython.display
from pathlib import Path
from tqdm import tqdm_notebook
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython import display as ipd
from mne.time_frequency import psd_multitaper
import mne

In [ ]:
DATA = Path('../input/freesound-audio-tagging-2019')
#PREPROCESSED = Path('../input/fat2019_prep_mels1')
PREPROCESSED = Path('work/fat2019_prep_mels1')
WORK = Path('work')
Path(PREPROCESSED).mkdir(exist_ok=True, parents=True)
Path(WORK).mkdir(exist_ok=True, parents=True)

CSV_TRN_CURATED = DATA/'train_curated.csv'
CSV_TRN_NOISY = DATA/'train_noisy.csv'
CSV_SUBMISSION = DATA/'sample_submission.csv'

TRN_CURATED = DATA/'train_curated'
TRN_NOISY = DATA/'train_noisy'
TEST = DATA/'test'

MELS_TRN_CURATED = PREPROCESSED/'mels_train_curated.pkl'
MELS_TRN_NOISY = PREPROCESSED/'mels_train_noisy.pkl'
MELS_TEST = PREPROCESSED/'mels_test.pkl'

MFCC_TRN_CURATED = PREPROCESSED/'mfcc_train_curated.pkl'
MFCC_TRN_NOISY = PREPROCESSED/'mfcc_train_noisy.pkl'
MFCC_TEST = PREPROCESSED/'mfcc_test.pkl'

CSV_TRN_NOISY_BEST50S = PREPROCESSED/'trn_noisy_best50s.csv'
MELS_TRN_NOISY_BEST50S = PREPROCESSED/'mels_trn_noisy_best50s.pkl'
MFCC_TRN_NOISY_BEST50S = PREPROCESSED/'mfcc_trn_noisy_best50s.pkl'

trn_curated_df = pd.read_csv(CSV_TRN_CURATED)
trn_noisy_df = pd.read_csv(CSV_TRN_NOISY)
test_df = pd.read_csv(CSV_SUBMISSION)

In [ ]:
def audio_analysis(x, draw_plot=True):
    
    # Obtain audio data
    rate, data = wavfile.read(TRN_CURATED/x)
    data = data.astype(float)
    data /= data.max()
    
    if draw_plot:
        # Plot
        plt.figure(figsize=(16,9))

        # Waveform
        ax = plt.subplot(311)
#         ax.set_title('{} / {} / {}'.format(x, x.labels, f'Curated={x.curated}'))
        ax.set_ylabel('Amplitude')    
        librosa.display.waveplot(data, sr=rate)

        # Spectrogram
        ax = plt.subplot(312)
        S = librosa.feature.melspectrogram(data, sr=rate, n_mels=128)
        log_S = librosa.power_to_db(S, ref=np.max)
        librosa.display.specshow(log_S, sr=rate, x_axis='time', y_axis='mel')
        ax.set_title('Mel power spectrogram ')
        plt.colorbar(format='%+02.0f dB')
        plt.tight_layout()

        # MFCC coefficient
        ax = plt.subplot(313)
        mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        librosa.display.specshow(delta2_mfcc)
        plt.ylabel('MFCC coeffs')
        plt.xlabel('Time')
        plt.title('MFCC')
        plt.colorbar()
        plt.tight_layout()
    
#     return ipd.Audio(filename=TRN_CURATED/x)

In [ ]:
# audio_analysis('02a5aae7.wav')

In [ ]:
rate, data = wavfile.read(TRN_CURATED/'02ddd8da.wav')
data = data.astype(float)

if 0 < len(data): 
    data, _ = librosa.effects.trim(data)  # trim, top_db=default(60)
    
# print(max(abs(data)), max(data))    
# print(np.where(abs(data) == max(abs(data))))
    
MaxAmp = np.where(abs(data) == max(abs(data)))[0]
MaxAmp = MaxAmp[np.random.randint(len(MaxAmp))]
print(MaxAmp)
# max_offset = len(data) - conf.samples_long
# #     print(max_offset)
# if max(abs(data)) == 0:
#     data = data[:conf.samples_long]
# elif (len(data)-index) < index:
#     offset = (np.random.randint(len(data)-index) if len(data)-index < max_offset else len(data)-index-np.random.randint(max_offset))
#     data_out = data[(index-(conf.samples_long-offset)):(index+offset)]
# else:
#     if index == 0:
#         offset = 0
#     elif index < max_offset:
#         offset = np.random.randint(index)
#     else:
#         offset = index-np.random.randint(max_offset)
#     data_out = data[(index-offset):(index+(conf.samples_long-offset))]

In [ ]:
%%time
drop_audio_file = ['f76181c4.wav', '77b925c2.wav', '6a1f682a.wav', 'c7db12aa.wav', '7752cc8a.wav','1d44b0bd.wav']
drop_index = trn_curated_df.loc[  trn_curated_df['fname'].isin( drop_audio_file ) ].index.values

for idx in drop_index:
    trn_curated_df = trn_curated_df.drop(idx)

In [ ]:
import librosa
import librosa.display
import random

def read_audio(conf, pathname, trim_long_data):
    rate, data = wavfile.read(pathname)
    data = data.astype(float)

    # workaround: 0 length causes error
    if 0 < len(data): 
        data, _ = librosa.effects.trim(data)  # trim, top_db=default(60)

    # make it unified length to SAMPLE_DATA_CNT
    if len(data) > conf.samples_long:             
        if trim_long_data:
#             print(pathname)
            # trim the audio where is nearby the maximun amplitude to a segment of 7 secs
            MaxAmp = np.where(abs(data) == max(abs(data)))[0]
            MaxAmp = MaxAmp[np.random.randint(len(MaxAmp))]
            max_offset = len(data) - conf.samples_long
            if max(abs(data)) == 0:
                data = data[:conf.samples_long]
            elif (len(data)-MaxAmp) < MaxAmp:
                offset = (np.random.randint(len(data)-MaxAmp) if len(data)-MaxAmp < max_offset else len(data)-MaxAmp-np.random.randint(max_offset))
                data = data[(MaxAmp-(conf.samples_long-offset)):(MaxAmp+offset)]
            else:
                if MaxAmp == 0:
                    offset = 0
                elif MaxAmp < max_offset:
                    offset = np.random.randint(MaxAmp)
                else:
                    offset = MaxAmp-np.random.randint(max_offset)
                data = data[(MaxAmp-offset):(MaxAmp+(conf.samples_long-offset))]
    else:                                     
        data = np.tile(data, math.floor(conf.samples/data.shape[0])) 
        if len(data) < conf.samples:
            max_offset = conf.samples-len(data)
            offset = np.random.randint(max_offset)
            data = np.pad(data, (offset, conf.samples-len(data)-offset), "constant")

    return data


def audio_to_mels_mfcc(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
#                                                  hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    mfcc = librosa.feature.mfcc(S=spectrogram, n_mfcc=13)
    mfcc = librosa.feature.delta(mfcc, order=2)
    return spectrogram, mfcc


def show_melspectrogram(conf, mels, title='Log-frequency power spectrogram'):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf.sampling_rate, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()


def read_as_mels_mfcc(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels, mfcc = audio_to_mels_mfcc(conf, x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
        show_melspectrogram(conf, mels)
    return mels, mfcc


class conf:
    sampling_rate = 44100
    duration = 3 # sec
    duration_long = 5
#     hop_length = 347*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    padmode = 'constant'
    samples = sampling_rate * duration
    samples_long = sampling_rate * (duration_long)


def get_default_conf():
    return conf

In [ ]:
# change the array to RBG scaler
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def convert_wav_to_image(df, source):
    X_mels = []
    X_mfcc = []
    for i, row in tqdm_notebook(df.iterrows()):
        x_mels, x_mfcc = read_as_mels_mfcc(conf, source/str(row.fname), trim_long_data=True)
        x_color = mono_to_color(x_mels)
        X_mels.append(x_color)
        X_mfcc.append(x_mfcc)
    return X_mels, X_mfcc


def save_as_pkl_binary(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
%%time
conf = get_default_conf()

def convert_dataset(df, source_folder, mels_filename, mfcc_filename):
    X_mels, X_mfcc = convert_wav_to_image(df, source=source_folder)
    save_as_pkl_binary(X_mels, mels_filename)
    save_as_pkl_binary(X_mfcc, mfcc_filename)
    print(f'Created {mels_filename}')
    print(f'Created {mfcc_filename}')
    return X_mels, X_mfcc

convert_dataset(trn_curated_df, TRN_CURATED, MELS_TRN_CURATED, MFCC_TRN_CURATED)

## Creating Best 50s

In [ ]:
df = trn_noisy_df.copy()
df['singled'] = ~df.labels.str.contains(',')
singles_df = df[df.singled]

In [ ]:
labels = singles_df.labels.unique()
labels, len(labels)

In [ ]:
idxes_best50s = np.array([random.choices(singles_df[(singles_df.labels == l)].index, k=50)
                          for l in labels]).ravel()
best50s_df = singles_df.loc[idxes_best50s]

In [ ]:
best50s_df.to_csv(CSV_TRN_NOISY_BEST50S, index=False)

### Now best 50s are selected

In [ ]:
X = convert_dataset(best50s_df, TRN_NOISY, MELS_TRN_NOISY_BEST50S, MFCC_TRN_NOISY_BEST50S)